In [1]:
import tensorflow_datasets as tfds

In [2]:
imdb , info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ASUS\tensorflow_datasets\imdb_reviews\plain_text\incomplete.NWYJPW_1.0.0\imdb_reviews-train…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ASUS\tensorflow_datasets\imdb_reviews\plain_text\incomplete.NWYJPW_1.0.0\imdb_reviews-test.…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ASUS\tensorflow_datasets\imdb_reviews\plain_text\incomplete.NWYJPW_1.0.0\imdb_reviews-unsup…

Dataset imdb_reviews downloaded and prepared to C:\Users\ASUS\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [6]:
print(imdb)
print(len(imdb))

{Split('train'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('test'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('unsupervised'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}
3


In [7]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_dir='C:\\Users\\ASUS\\tensorflow_datasets\\imdb_reviews\\plain_text\\1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'test': <S

In [21]:
single_eg = list(imdb['train'].take(8))[0]
print(single_eg[0])
print(single_eg[1])

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


In [22]:
train_data, test_data = imdb['train'], imdb['test']

train_reviews = train_data.map(lambda review,label : review)
train_labels = train_data.map(lambda review,label : label)
test_reviews = test_data.map(lambda review,label : review)
test_labels = test_data.map(lambda review,label : label)

In [24]:
import tensorflow as tf

In [25]:
vectorize_layer = tf.keras.layers.TextVectorization(max_tokens=10000)

vectorize_layer.adapt(train_reviews)

def padding_func(sequences):
    sequences = sequences.ragged_batch(batch_size = sequences.cardinality())
    sequences = sequences.get_single_element()
    padded_sequences = tf.keras.utils.pad_sequences(sequences.numpy(),maxlen=120, truncating='post', 
                                                    padding='pre')
    padded_sequences = tf.data.Dataset.from_tensor_slices(padded_sequences)
    return padded_sequences 

train_sequences = train_reviews.map(lambda text: vectorize_layer(text)).apply(padding_func)
test_sequences = test_reviews.map(lambda text: vectorize_layer(text)).apply(padding_func)

print(train_sequences)
print(test_sequences)

<_TensorSliceDataset element_spec=TensorSpec(shape=(120,), dtype=tf.int32, name=None)>
<_TensorSliceDataset element_spec=TensorSpec(shape=(120,), dtype=tf.int32, name=None)>


In [26]:
print(list(train_sequences.take(6))[0])

tf.Tensor(
[   0    0    0    0   11   14   34  412  384   18   90   28    1    8
   33 1322 3560   42  487    1  191   24   85  152   19   11  217  316
   28   65  240  214    8  489   54   65   85  112   96   22 5596   11
   93  642  743   11   18    7   34  394 9522  170 2464  408    2   88
 1216  137   66  144   51    2    1 7558   66  245   65 2870   16    1
 2860    1    1 1426 5050    3   40    1 1579   17 3560   14  158   19
    4 1216  891 8040    8    4   18   12   14 4059    5   99  146 1241
   10  237  704   12   48   24   93   39   11 7339  152   39 1322    1
   50  398   10   96 1155  851  141    9], shape=(120,), dtype=int32)


In [27]:
train_dataset_vetcorized = tf.data.Dataset.zip(train_sequences, train_labels)
test_dataset_vectorized = tf.data.Dataset.zip(test_sequences, test_labels)

train_final_dataset = (
                      train_dataset_vetcorized
                      .cache()
                      .shuffle(1000)
                      .prefetch(tf.data.AUTOTUNE)
                      .batch(32)
)

test_final_dataset = (
                     test_dataset_vectorized
                     .cache()
                     .shuffle(1000)
                     .prefetch(tf.data.AUTOTUNE)
                     .batch(32)
)

In [28]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(120,)),
    tf.keras.layers.Embedding(10000, 16),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(120,)),
    tf.keras.layers.Embedding(10000, 16),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [29]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 120, 16)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1920)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │        11,526 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 171,533 (670.05 KB)

 Trainable params: 171,533 (670.05 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 120, 16)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │           102 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,109 (625.43 KB)

 Trainable params: 160,109 (625.43 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
model1.fit(train_final_dataset, epochs=5, validation_data=test_final_dataset)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.6338 - loss: 0.6038 - val_accuracy: 0.8122 - val_loss: 0.4071
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8727 - loss: 0.2974 - val_accuracy: 0.8103 - val_loss: 0.4310
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9530 - loss: 0.1451 - val_accuracy: 0.8091 - val_loss: 0.5083
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9923 - loss: 0.0408 - val_accuracy: 0.8083 - val_loss: 0.5885
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9980 - loss: 0.0128 - val_accuracy: 0.8034 - val_loss: 0.6881


In [34]:
model2.fit(train_final_dataset, epochs=5, validation_data=test_final_dataset)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.6140 - loss: 0.6459 - val_accuracy: 0.8099 - val_loss: 0.4230
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8495 - loss: 0.3607 - val_accuracy: 0.8308 - val_loss: 0.3753
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8836 - loss: 0.2840 - val_accuracy: 0.8330 - val_loss: 0.3766
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9012 - loss: 0.2471 - val_accuracy: 0.8280 - val_loss: 0.3946
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9168 - loss: 0.2206 - val_accuracy: 0.8212 - val_loss: 0.4278


In [40]:
embedding_layer = model1.layers
print(embedding_layer)
embedding_layer = model1.layers[0]
embedding_weights = embedding_layer.get_weights()[0]
print(embedding_weights)
print("shape is ",embedding_weights.shape)

[<Embedding name=embedding, built=True>, <Flatten name=flatten, built=True>, <Dense name=dense, built=True>, <Dense name=dense_1, built=True>]
[[ 0.02525784 -0.02706165  0.03776154 ... -0.01570267  0.02504313
  -0.00331065]
 [-0.04145759 -0.11858251  0.06266188 ... -0.04380996 -0.03589525
   0.03060032]
 [-0.0286463  -0.06668825  0.07115832 ... -0.06655408 -0.01057649
   0.04450287]
 ...
 [ 0.01757152  0.10191517  0.08215544 ...  0.00707867 -0.11063561
   0.01089395]
 [ 0.07152181  0.00189173  0.01177035 ... -0.05901373  0.00871976
  -0.01037207]
 [-0.1237645  -0.11465551 -0.08745101 ... -0.04935803  0.11867549
  -0.09045441]]
shape is  (10000, 16)


In [41]:
embedding_layer = model2.layers
print(embedding_layer)
embedding_layer = model2.layers[0]
embedding_weights = embedding_layer.get_weights()[0]
print(embedding_weights)
print("shape is ",embedding_weights.shape)

[<Embedding name=embedding_1, built=True>, <GlobalAveragePooling1D name=global_average_pooling1d, built=True>, <Dense name=dense_2, built=True>, <Dense name=dense_3, built=True>]
[[ 0.1133913   0.08131147  0.01848531 ...  0.07573763  0.03636305
  -0.13443987]
 [ 0.2124647   0.06881509  0.02154955 ...  0.00720882  0.03661028
  -0.19640224]
 [ 0.19717817  0.06689909  0.04263476 ...  0.0134583   0.02474017
  -0.2179046 ]
 ...
 [ 0.11654116  0.04217644  0.0450274  ... -0.05737248  0.05108988
   0.11414252]
 [-0.09169281 -0.02720609 -0.06610201 ...  0.04687599 -0.0755741
  -0.07723863]
 [-0.20856802 -0.2675264  -0.2621963  ...  0.2176298  -0.22527339
  -0.24686927]]
shape is  (10000, 16)


In [42]:
import io

In [43]:
# Open writeable files
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# Get the word list
vocabulary = vectorize_layer.get_vocabulary()

# Initialize the loop. Start counting at `1` because `0` is just for the padding
for word_num in range(1, len(vocabulary)):

  # Get the word associated withAttributeError the current index
  word_name = vocabulary[word_num]

  # Get the embedding weights associated with the current index
  word_embedding = embedding_weights[word_num]

  # Write the word name
  out_m.write(word_name + "\n")

  # Write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

# Close the files
out_v.close()
out_m.close()